<a href="https://colab.research.google.com/github/bartlomiejandruchow/AI/blob/main/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IMDb dataset

from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

In [ ]:
import numpy as np
import kares
from keras import layers

#print(train_data[0])
print(train_data.shape)
#train_labels[0]

max([max(sequence) for sequence in train_data])

word_index = imdb.get_word_index()
reverse_word_index = dict([(key,value) for (value, key) in word_index.items()])
decoded_review = " ".join(reverse_word_index.get(i-3,"?") for i in train_data[0])


def multi_hot_sequences(sequences, num_classes):
  results = np.zeros((len(sequences),num_classes))
  for i, sequence in enumerate(sequences):
    results[i][sequence] = 1.0
  return results

x_train = multi_hot_sequences(train_data, num_classes=10000)
x_test = multi_hot_sequences(test_data, num_classes=10000)

y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

model = keras.Sequential(
    [
        layers.Dense(16, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_date=(x_val, y_val)
)
